In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import re
import string

from bs4 import BeautifulSoup

In [279]:
df = pd.read_csv('../data/yoga.csv')

In [280]:
df['description'].fillna('<no description>', inplace=True)
df['tags'].fillna('<no tags>', inplace=True)

In [4]:
def find_nonstr(df):
    lst=[]
    for index, desc in enumerate(df):
        if type(desc) != str:
            lst.append(index)
    return lst

In [158]:
def clean_text(text):
    head, sep, tail = text.partition('\n\n')
    no_emo = head.encode('ascii', 'ignore').decode('ascii')
    no_punc = re.sub('[,\.!?]', '', no_emo)
    no_special_char = re.sub(r"(@[A-Za-z0-9]+)|(\w+:\/\/\S+)|^rt|http.+?|\n", "", no_punc)
    return no_special_char.lower()

In [159]:
to_clean = ['title', 'description', 'tags']

In [160]:
for column in to_clean:
    df[column] = df[column].apply(clean_text)

In [281]:
def clean_text(text):
    head, sep, tail = text.partition('\n\n')
    return head

In [282]:
clean_text(df['description'][0])

'De-stress with this 20 minute calming yoga routine that includes light and easy full body stretches for stress relief and anxiety.'

---

In [ ]:
# lda with gensim

In [106]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import random
import string

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [164]:
df_1000 = df.sample(1000, random_state=1).reset_index()

In [165]:
df_1000['documents'] = df_1000['title'] + ' ' + df_1000['description'] + ' ' + df_1000['tags']

__LDA with gensim__

In [134]:
#gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim.corpora as corpora
from gensim.models import CoherenceModel

#nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/joshchow/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [167]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = df_1000['documents'].values.tolist()
data_words = list(sent_to_words(data))

In [233]:
channel_stopwords = list(sent_to_words(df_1000['channelTitle'].unique().tolist()))
all_stopwords = []
for words in channel_stopwords:
    for word in words:
        all_stopwords.append(word)

In [237]:
# custom stop_words
all_stopwords.extend(['yoga', 'movie', 'youtube', 'http', 'hour', 'min', 'minute', 'donate', 'practice', 'donation'])
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(all_stopwords)
# Define functions for stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [238]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['support', 'visit', 'sure', 'enter', 'support', 'support']]


In [239]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 1)]]


In [240]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [241]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.019*"exercise" + 0.014*"pose" + 0.009*"advanced" + 0.009*"video" + '
  '0.009*"help" + 0.009*"level" + 0.008*"technique" + 0.007*"beginner" + '
  '0.007*"nyk" + 0.007*"step"'),
 (1,
  '0.033*"trimester" + 0.033*"prenatal" + 0.021*"chakra" + 0.020*"day" + '
  '0.014*"pregnant" + 0.012*"bowl" + 0.011*"third" + 0.010*"leg" + '
  '0.009*"help" + 0.009*"second"'),
 (2,
  '0.033*"beginner" + 0.027*"weight" + 0.025*"loss" + 0.024*"full" + '
  '0.020*"vinyasa" + 0.018*"intermediate" + 0.014*"total" + 0.014*"minute" + '
  '0.013*"power" + 0.011*"video"'),
 (3,
  '0.064*"morning" + 0.038*"full" + 0.025*"restorative" + 0.023*"stretch" + '
  '0.019*"beginner" + 0.012*"routine" + 0.012*"prop" + 0.012*"calm" + '
  '0.009*"exercise" + 0.009*"day"'),
 (4,
  '0.042*"stretch" + 0.034*"morning" + 0.019*"full" + 0.013*"vinyasa" + '
  '0.012*"routine" + 0.011*"jump" + 0.010*"open" + 0.010*"video" + '
  '0.009*"sequence" + 0.008*"pose"'),
 (5,
  '0.057*"pain" + 0.055*"back" + 0.037*"restorative" +

In [242]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.41029466836682227


In [243]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [249]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()





  0%|          | 0/540 [00:00<?, ?it/s]



  0%|          | 1/540 [00:01<13:39,  1.52s/it]



  0%|          | 2/540 [00:02<13:11,  1.47s/it]



  1%|          | 3/540 [00:04<12:42,  1.42s/it]



  1%|          | 4/540 [00:05<12:47,  1.43s/it]



  1%|          | 5/540 [00:07<12:35,  1.41s/it]



  1%|          | 6/540 [00:08<12:42,  1.43s/it]



  1%|▏         | 7/540 [00:10<13:03,  1.47s/it]



  1%|▏         | 8/540 [00:11<13:58,  1.58s/it]



  2%|▏         | 9/540 [00:13<14:33,  1.64s/it]



  2%|▏         | 10/540 [00:15<14:43,  1.67s/it]



  2%|▏         | 11/540 [00:17<15:55,  1.81s/it]



  2%|▏         | 12/540 [00:19<17:23,  1.98s/it]



  2%|▏         | 13/540 [00:22<18:14,  2.08s/it]



  3%|▎         | 14/540 [00:24<18:08,  2.07s/it]



  3%|▎         | 15/540 [00:26<17:33,  2.01s/it]



  3%|▎         | 16/540 [00:27<17:09,  1.97s/it]



  3%|▎         | 17/540 [00:29<16:29,  1.89s/it]



  3%|▎         | 18/540 [00:31<16:31,  1.90s/it]



  4%|▎         | 19/540 [0

In [251]:
results = pd.read_csv('lda_tuning_results.csv')

In [275]:
results[results['Validation_Set']=='100% Corpus'].describe()

,Topics,Coherence
count,270.000000,270.000000
mean,6.000000,0.410929
std,2.586784,0.025537
min,2.000000,0.348852
25%,4.000000,0.396501
50%,6.000000,0.410013
75%,8.000000,0.426318
max,10.000000,0.480207


In [255]:
results[results['Coherence']>0.48]

,Validation_Set,Topics,Alpha,Beta,Coherence
503,100% Corpus,9,symmetric,0.9099999999999999,0.480207


In [256]:
# final model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha='symmetric',
                                           eta=0.9)

In [260]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.47855287601674745


In [261]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.128393  0.034402       1        1  35.224013
2     -0.030738 -0.071727       2        1  16.136045
0     -0.007769  0.018531       3        1  12.631719
1      0.055378  0.045022       4        1  10.103322
7     -0.004210 -0.020789       5        1   8.677704
3      0.015344  0.004291       6        1   5.612996
4      0.020672 -0.005303       7        1   4.314460
8      0.032305 -0.001928       8        1   4.285012
6      0.047411 -0.002499       9        1   3.014728, topic_info=        Term        Freq       Total Category  logprob  loglift
31   morning  102.000000  102.000000  Default  30.0000  30.0000
47    chakra   83.000000   83.000000  Default  29.0000  29.0000
108     back  188.000000  188.000000  Default  28.0000  28.0000
121     pain  184.000000  184.000000  Default  27.0000  27.0000
130  stretch  252.000000  252.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
104    short    0.943630   32.689428   Topic9  -6.5081  -0.0434
36   provide    0.854744   20.103066   Topic9  -6.6070   0.3438
179     feel    0.975347   67.676176   Topic9  -6.4751  -0.7380
389     calm    0.834575   27.319359   Topic9  -6.6309   0.0132
41      self    0.821232   35.118921   Topic9  -6.6470  -0.2540

[507 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
1273      1  0.278393          abuse
1273      8  0.278393          abuse
44        1  0.097927       activate
44        2  0.195855       activate
44        3  0.097927       activate
...     ...       ...            ...
258       8  0.137742           year
1879      1  0.256708  yogachallenge
1879      6  0.256708  yogachallenge
3401      1  0.211771         yogale
3401      3  0.423542         yogale

[1414 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 1, 2, 8, 4, 5, 9, 7])

In [271]:
lda_model.save('../data/lda')

[(5, 0.031873174)]

In [277]:
from gensim import similarities
def get_similarity(lda, query_vector):
    index = similarities.MatrixSimilarity(lda[corpus])
    sims = index[query_vector]
    return sims

In [1]:
dir(lda_model)

NameError: name 'lda_model' is not defined